In [1]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# Function to load images and masks
def load_images_and_masks(image_folder, mask_folder, img_size):
    images = []
    masks = []

    for filename in os.listdir(image_folder):
        img_path = os.path.join(image_folder, filename)
        mask_path = os.path.join(mask_folder, filename)
        
        print(f"Checking {filename}:")
        print(f"Image path: {img_path}")
        print(f"Mask path: {mask_path}")

        if os.path.exists(img_path) and os.path.exists(mask_path):
            image = cv2.imread(img_path)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            
            if image is not None and mask is not None:
                # Resize image and mask
                image = cv2.resize(image, (img_size, img_size)) / 255.0
                mask = cv2.resize(mask, (img_size, img_size)) / 255.0
                mask = np.expand_dims(mask, axis=-1)

                images.append(image)
                masks.append(mask)
            else:
                print(f"Failed to read image or mask for {filename}")

    print(f"Loaded {len(images)} images and {len(masks)} masks")
    return np.array(images), np.array(masks)

# Paths to your image and mask folders
image_folder = r'C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\image_folder'
mask_folder = r'C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\mask_folder'
img_size = 256

# Load and preprocess the data
X, y = load_images_and_masks(image_folder, mask_folder, img_size)

# Check if images and masks were loaded correctly
print(f"Number of images: {X.shape[0]}")
print(f"Number of masks: {y.shape[0]}")

# Split the data into training and validation sets
if X.shape[0] > 0 and y.shape[0] > 0:
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Function to create the U-Net model
    def unet_model(input_shape, num_classes):
        inputs = Input(input_shape)

        # Encoder
        c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
        c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
        p1 = MaxPooling2D((2, 2))(c1)
        p1 = Dropout(0.1)(p1)

        c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
        c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
        p2 = MaxPooling2D((2, 2))(c2)
        p2 = Dropout(0.1)(p2)

        c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
        c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
        p3 = MaxPooling2D((2, 2))(c3)
        p3 = Dropout(0.2)(p3)

        c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
        c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
        p4 = MaxPooling2D((2, 2))(c4)
        p4 = Dropout(0.2)(p4)

        c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
        c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)
        c5 = Dropout(0.3)(c5)

        # Decoder
        u6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
        u6 = Concatenate()([u6, c4])
        c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
        c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)
        c6 = Dropout(0.2)(c6)

        u7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
        u7 = Concatenate()([u7, c3])
        c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
        c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)
        c7 = Dropout(0.2)(c7)

        u8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
        u8 = Concatenate()([u8, c2])
        c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
        c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)
        c8 = Dropout(0.1)(c8)

        u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
        u9 = Concatenate()([u9, c1])
        c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
        c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)
        c9 = Dropout(0.1)(c9)

        outputs = Conv2D(num_classes, (1, 1), activation='sigmoid')(c9)

        model = Model(inputs, outputs)
        return model

    # Create and compile the model
    input_shape = (img_size, img_size, 3)
    num_classes = 1  # For binary segmentation (object vs background)

    model = unet_model(input_shape, num_classes)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


    # Print the model summary
    model.summary()

    # Train the model
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10,batch_size=8)

    # Save the model
    model.save('unet_cinnamon_segmentation.h5')

    # Load the model
    model = load_model('unet_cinnamon_segmentation.h5')

    # Predict masks
   # y_pred = model.predict(X_val)

    # Threshold predictions to get binary masks
    #y_pred_thresh = (y_pred > 0.5).astype(np.uint8)

    # Compute metrics
    #y_true_flat = y_val.flatten()
    #y_pred_flat = y_pred_thresh.flatten()

    #precision = precision_score(y_true_flat, y_pred_flat)
    #recall = recall_score(y_true_flat, y_pred_flat)
    #f1 = f1_score(y_true_flat, y_pred_flat)

   # print(f"Precision: {precision:.4f}")
    #print(f"Recall: {recall:.4f}")
    #print(f"F1-Score: {f1:.4f}")

    # Dice Coefficient
   # def dice_coefficient(y_true, y_pred, smooth=1):
        #intersection = np.sum(y_true * y_pred)
       # return (2. * intersection + smooth) / (np.sum(y_true) + np.sum(y_pred) + smooth)

   # dice = dice_coefficient(y_val, y_pred_thresh)
    #print(f"Dice Coefficient: {dice:.4f}")

    # Intersection over Union (IoU)
    #def iou(y_true, y_pred):
       # intersection = np.sum(y_true * y_pred)
        #union = np.sum(y_true) + np.sum(y_pred) - intersection
        #return intersection / union

   # iou_score = iou(y_val, y_pred_thresh)
    #print(f"IoU: {iou_score:.4f}")
#else:
    #print("No images or masks loaded.")


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\User\\Desktop\\back up\\my doc\\RESEARCH\\CINNAMON CODES\\image_folder'

Checking IMG_20230928_114216.png:
Image path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\image_folder\IMG_20230928_114216.png
Mask path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\mask_folder\IMG_20230928_114216.png
Checking IMG_20230928_114232.png:
Image path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\image_folder\IMG_20230928_114232.png
Mask path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\mask_folder\IMG_20230928_114232.png
Checking IMG_20230928_114306.png:
Image path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\image_folder\IMG_20230928_114306.png
Mask path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\mask_folder\IMG_20230928_114306.png
Checking IMG_20230928_114329.png:
Image path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\image_folder\IMG_20230928_114329.png
Mask path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\mask_folder\IMG_20230928_114

Checking IMG_20230928_115730291.png:
Image path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\image_folder\IMG_20230928_115730291.png
Mask path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\mask_folder\IMG_20230928_115730291.png
Checking IMG_20230928_115816.png:
Image path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\image_folder\IMG_20230928_115816.png
Mask path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\mask_folder\IMG_20230928_115816.png
Checking IMG_20230928_115847.png:
Image path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\image_folder\IMG_20230928_115847.png
Mask path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\mask_folder\IMG_20230928_115847.png
Checking IMG_20230928_115853.png:
Image path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\image_folder\IMG_20230928_115853.png
Mask path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\mask_folder\IMG_202

Checking IMG_20230928_133357.png:
Image path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\image_folder\IMG_20230928_133357.png
Mask path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\mask_folder\IMG_20230928_133357.png
Checking IMG_20230928_133406.png:
Image path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\image_folder\IMG_20230928_133406.png
Mask path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\mask_folder\IMG_20230928_133406.png
Checking IMG_20230928_133425.png:
Image path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\image_folder\IMG_20230928_133425.png
Mask path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\mask_folder\IMG_20230928_133425.png
Checking IMG_20230928_133520.png:
Image path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\image_folder\IMG_20230928_133520.png
Mask path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\mask_folder\IMG_20230928_133

Checking IMG_20230928_135515837.png:
Image path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\image_folder\IMG_20230928_135515837.png
Mask path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\mask_folder\IMG_20230928_135515837.png
Checking IMG_20230928_135536.png:
Image path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\image_folder\IMG_20230928_135536.png
Mask path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\mask_folder\IMG_20230928_135536.png
Checking IMG_20230928_135653.png:
Image path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\image_folder\IMG_20230928_135653.png
Mask path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\mask_folder\IMG_20230928_135653.png
Checking IMG_20230928_135659.png:
Image path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\image_folder\IMG_20230928_135659.png
Mask path: C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\mask_folder\IMG_202

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │     36,928 │ conv2d[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128, 128,  │          0 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │     73,856 │ dropout[0][0]     │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │    147,584 │ conv2d_2[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64, 64,    │          0 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │    295,168 │ dropout_1[0][0]   │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 64, 64,    │    590,080 │ conv2d_4[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 32, 32,    │          0 │ conv2d_5[0][0]    │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 32, 32,    │          0 │ max_pooling2d_2[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 32, 32,    │  1,180,160 │ dropout_2[0][0]   │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 32, 32,    │  2,359,808 │ conv2d_6[0][0]    │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 16, 16,    │          0 │ conv2d_7[0][0]    │
│ (MaxPooling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 16, 16,    │          0 │ max_pooling2d_3[

 Total params: 31,031,745 (118.38 MB)

 Trainable params: 31,031,745 (118.38 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 4188s 44s/step - accuracy: 0.7236 - loss: 0.4723 - val_accuracy: 0.7261 - val_loss: 0.3780
Epoch 2/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 3985s 42s/step - accuracy: 0.6890 - loss: 1.2452 - val_accuracy: 0.7261 - val_loss: 0.5031
Epoch 3/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 4002s 43s/step - accuracy: 0.7180 - loss: 0.4526 - val_accuracy: 0.7053 - val_loss: 0.2433
Epoch 4/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 23247s 250s/step - accuracy: 0.7006 - loss: 0.2546 - val_accuracy: 0.7124 - val_loss: 0.2227
Epoch 5/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 3964s 42s/step - accuracy: 0.7056 - loss: 0.2267 - val_accuracy: 0.7134 - val_loss: 0.2084
Epoch 6/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 3956s 42s/step - accuracy: 0.7050 - loss: 0.2089 - val_accuracy: 0.7165 - val_loss: 0.2133
Epoch 7/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 4469s 48s/step - accuracy: 0.7025 - loss: 0.2086 - val_accuracy: 0.7141 - val_loss: 0.1975
Epoch 8/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 3668s 39s/step - accuracy: 0.7128 - loss: 0.1989 - val_ac

In [ ]:
y_pred = model.predict(X_val)
# Ensure that y_val is binary
 #Threshold predictions to get binary masks
y_pred_thresh = (y_pred > 0.5).astype(np.uint8)
y_val_thresh = (y_val > 0.5).astype(np.uint8)

# Flatten the arrays for metric calculation
y_true_flat = y_val_thresh.flatten()
y_pred_flat = y_pred_thresh.flatten()

# Compute metrics
precision = precision_score(y_true_flat, y_pred_flat)
recall = recall_score(y_true_flat, y_pred_flat)
f1 = f1_score(y_true_flat, y_pred_flat)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Dice Coefficient
def dice_coefficient(y_true, y_pred, smooth=1):
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection + smooth) / (np.sum(y_true) + np.sum(y_pred) + smooth)

dice = dice_coefficient(y_val_thresh, y_pred_thresh)
print(f"Dice Coefficient: {dice:.4f}")

# Intersection over Union (IoU)
def iou(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    union = np.sum(y_true) + np.sum(y_pred) - intersection
    return intersection / union

iou_score = iou(y_val_thresh, y_pred_thresh)
print(f"IoU: {iou_score:.4f}")


In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

# Function to preprocess image
def preprocess_image(image_path, img_size):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (img_size, img_size)) / 255.0
    image = np.expand_dims(image, axis=0)
    return image

# Load the trained model
model = load_model('unet_cinnamon_segmentation.h5')

# Path to the new image
new_image_path = r'C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\test_image2.jpg'

# Preprocess the new image
img_size = 256  # Same size used during training
new_image = preprocess_image(new_image_path, img_size)

# Predict the mask
predicted_mask = model.predict(new_image)

# Threshold the predicted mask
predicted_mask = (predicted_mask > 0.5).astype(np.uint8)

# Remove the batch dimension
predicted_mask = np.squeeze(predicted_mask)

# Plot the original image and the predicted mask
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Original Image
original_image = cv2.imread(new_image_path)
original_image = cv2.resize(original_image, (img_size, img_size))
ax[0].imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
ax[0].set_title('Original Image')
ax[0].axis('off')

# Predicted Mask
ax[1].imshow(predicted_mask, cmap='gray')
ax[1].set_title('Predicted Mask')
ax[1].axis('off')

plt.show()


In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

# Function to preprocess image
def preprocess_image(image_path, img_size):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (img_size, img_size)) / 255.0
    image = np.expand_dims(image, axis=0)
    return image

# Load the trained model
model = load_model('unet_cinnamon_segmentation.h5')

# Path to the folder containing new images
image_folder_path = r'C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\Test_images'
img_size = 256  # Same size used during training

# Process each image in the folder
for image_file in os.listdir(image_folder_path):
    image_path = os.path.join(image_folder_path, image_file)

    # Preprocess the image
    new_image = preprocess_image(image_path, img_size)

    # Predict the mask
    predicted_mask = model.predict(new_image)

    # Threshold the predicted mask
    predicted_mask = (predicted_mask > 0.5).astype(np.uint8)

    # Remove the batch dimension
    predicted_mask = np.squeeze(predicted_mask)

    # Plot the original image and the predicted mask
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    # Original Image
    original_image = cv2.imread(image_path)
    original_image = cv2.resize(original_image, (img_size, img_size))
    ax[0].imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
    ax[0].set_title('Original Image')
    ax[0].axis('off')

    # Predicted Mask
    ax[1].imshow(predicted_mask, cmap='gray')
    ax[1].set_title('Predicted Mask')
    ax[1].axis('off')

    plt.show()
